In [125]:
import pandas as pd
import numpy as np

# df is Travel day trip dataframe
df = pd.read_csv('./RTS Public Files/trip.csv')

# df_p is each person's dataframe
df_p = pd.read_csv('./RTS Public Files/person.csv')

#df_h is for household
df_h = pd.read_csv('./RTS Public Files/household.csv')

In [126]:
# renaming columns like NHTS data
df.rename(columns={'HOUSEHOLD_ID': 'HOUSEID', 'PERSON_ID': 'PERSONID','TRIPNO': 'TDTRPNUM','O_PURPOSE': 'WHYFROM', 'D_PURPOSE': 'WHYTO',
                   'REPORTED_TRAVEL_TIME': 'TRVL_MIN','DISTANCE': 'TRPMILES','DEPARTURE_TIME_HHMM': 'STRTTIME','ARRIVAL_TIME_HHMM': 'ENDTIME'}, inplace=True)
df_p.rename(columns={'HOUSEHOLD_ID': 'HOUSEID', 'PERSON_ID': 'PERSONID','AGE': 'R_AGE',
                     'GENDER': 'R_SEX', 'EMPLOYMENT_STATUS':'wkstat'}, inplace=True)
df_h.rename(columns={'HOUSEHOLD_ID': 'HOUSEID', 'HHSIZE': 'HHSIZE', 'NUMVEHICLE': 'HHVEHCNT', 'HH_INCOME_DETAILED':'HHFAMINC', 'HOME_TPB_TAZ':'home_tpb_taz',
                     'NUMWORKERS':'hhwrk'}, inplace=True)

In [127]:
# variables we wish to keep
df = df[['HOUSEID', 'PERSONID','WHYFROM' , 'WHYTO', 'TRPMILES', 'STRTTIME','ENDTIME',
                           'TDTRPNUM', 'TRVL_MIN']]
df_p = df_p[['HOUSEID', 'PERSONID', 'R_SEX', 'R_AGE', 'wkstat']]
df_h = df_h[['HOUSEID', 'HHFAMINC','HHSIZE','HHVEHCNT', 'home_tpb_taz', 'hhwrk']]

In [128]:
# Valid Values	1	Less than $15,000
# 	2	$15,000-$24,999
# 	3	$25,000-$34,999
# 	4	$35,000-$49,999
# 	5	$50,000-$74,999
# 	6	$75,000-$99,999
# 	7	$100,000-$149,999
# 	8	$150,000 or more



In [129]:
df_p = df_p.join(df_h.set_index('HOUSEID'), on='HOUSEID')

In [130]:
df_p['HHFAMINC'].replace([1, 2, 3, 4], 2, inplace=True) 
df_p['HHFAMINC'].replace([5, 6, 7], 3, inplace=True) 
df_p['HHFAMINC'].replace([8], 4, inplace=True) 

In [131]:
#changing time to military format
df['STRTTIME'] = df['STRTTIME'].str.replace(':', '')
df['ENDTIME'] = df['ENDTIME'].str.replace(':', '')
df[['STRTTIME','ENDTIME']] = df[['STRTTIME','ENDTIME']].astype(int)

In [132]:
# making DWELTIME colum which show the time spent at the destination

def get_minutes(s):
    return s//100 * 60 + s % 100

df['DWELTIME'] = (get_minutes(df.STRTTIME)
                     .groupby([df.HOUSEID, df.PERSONID])
                     .shift(-1)
                     .sub(get_minutes(df.ENDTIME)).fillna(0)
                  )

## replacing values in columns 

In [133]:
# change the items in the given list with 11 and
work_trips = [2, 3] # = 2
education_trips = [8]
other_trips = [1, 4, 5, 7, 8,9,11,13, 16, 17, 18, 77, 99] # = 4
# add 1 later to the other category

#regrouping purpose classification
df['WHYTO'].replace(work_trips, 2, inplace=True)
df['WHYTO'].replace(education_trips, 3, inplace=True)
df['WHYTO'].replace(other_trips, 4, inplace=True)

#regrouping purpose classification
'''df['WHYFROM'].replace(work_education_trips, 2, inplace=True)
df['WHYFROM'].replace(shop_trips, 3, inplace=True)
df['WHYFROM'].replace(other_trips, 4, inplace=True)'''

"df['WHYFROM'].replace(work_education_trips, 2, inplace=True)\ndf['WHYFROM'].replace(shop_trips, 3, inplace=True)\ndf['WHYFROM'].replace(other_trips, 4, inplace=True)"

In [134]:
'''#making home-based and non-homebased column
def hb_nhb(whyfrom, whyto):
  if whyfrom != 1 and whyto != 1:
    return 'NHB'
  else:
    if whyfrom == 2 or whyto == 2:
      return 'HBW'
    elif whyfrom == 3 or whyto == 3:
      return 'HBSHOP'
    elif whyfrom == 4 or whyto == 4:
      return 'HBO'

df['TRIPPURP'] = df.apply(lambda row:hb_nhb(row['WHYFROM'], row['WHYTO']), axis=1 )'''

"#making home-based and non-homebased column\ndef hb_nhb(whyfrom, whyto):\n  if whyfrom != 1 and whyto != 1:\n    return 'NHB'\n  else:\n    if whyfrom == 2 or whyto == 2:\n      return 'HBW'\n    elif whyfrom == 3 or whyto == 3:\n      return 'HBSHOP'\n    elif whyfrom == 4 or whyto == 4:\n      return 'HBO'\n\ndf['TRIPPURP'] = df.apply(lambda row:hb_nhb(row['WHYFROM'], row['WHYTO']), axis=1 )"

In [135]:
'''#changing home purpose to other
df['WHYTO'].replace(1, 4, inplace=True)'''

"#changing home purpose to other\ndf['WHYTO'].replace(1, 4, inplace=True)"

In [136]:
# changing gender classification categories to 0(male) and 1(female)
df_p['R_SEX'].replace(1, 0, inplace=True)
df_p['R_SEX'].replace(2, 1, inplace=True)

In [137]:
# change the items in the given list with 11 and
# = 3  = parent
# = 2 = child
# = 1 = other
# = 0 = reference person
#other = [1, 4, 5, 6, 7, 9 ] # = 1

#regrouping purpose classification
#df_p['relate'].replace(other, 1, inplace=True)
#df_p['relate'].value_counts()

In [138]:
#defining 4 time of the day intervals
# 1 = A.M
# 2 = P.M
# 3 = mid_day
# 4 = night

df['TRVL_DAY_TIME'] = np.where((df['STRTTIME'] >= 500) & (df['STRTTIME'] <= 1000), 1, 
                  np.where((df['STRTTIME'] >= 1001) & (df['STRTTIME'] <= 1530), 2,
                  np.where((df['STRTTIME'] >= 1531) & (df['STRTTIME'] <= 2000), 3, 4)))

In [139]:
#changing not available data for these two to 0 because we don't want to delete them
df['TRPMILES'].replace([-9], np.nan, inplace=True)

In [140]:
'''#counting the number of home based and non home based trips
counts = df.groupby(["HOUSEID", "PERSONID", "TRIPPURP"]).apply(len).unstack(fill_value=0)
counts.columns = counts.columns.map(lambda x: f"{x}")
df_p = df_p.merge(counts, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.fillna(0, inplace=True)'''

'#counting the number of home based and non home based trips\ncounts = df.groupby(["HOUSEID", "PERSONID", "TRIPPURP"]).apply(len).unstack(fill_value=0)\ncounts.columns = counts.columns.map(lambda x: f"{x}")\ndf_p = df_p.merge(counts, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)\ndf_p.fillna(0, inplace=True)'

In [141]:
#df['pmode'].replace([2,3], 2, inplace=True)
#df['pmode'].replace([6,7,9], 3, inplace=True)

In [142]:
# defining 3 columns that shows the number of trips by category
# counts12 = df.groupby(["HOUSEID", "PERSONID", "pmode"]).apply(len).unstack(fill_value=0)
# counts12.columns = counts12.columns.map(lambda x: f"pmode{x}")
# df_p = df_p.merge(counts12, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
# df_p.rename(columns={'pmode1': 'transit', 'pmode2': 'auto', 'pmode3': 'other'}, inplace=True)

In [143]:
# defining 3 columns that shows the number of trips by category
counts = df.groupby(["HOUSEID", "PERSONID", "WHYTO"]).apply(len).unstack(fill_value=0)
counts.columns = counts.columns.map(lambda x: f"WHYTO{x}")
df_p = df_p.merge(counts, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'WHYTO2': 'WORK', 'WHYTO3': 'EDUCATION', 'WHYTO4': 'OTHER'}, inplace=True)

In [144]:
# making the columns for each daytime and merging it to the person dataframe
counts1 = df.groupby(["HOUSEID", "PERSONID", "TRVL_DAY_TIME"]).apply(len).unstack(fill_value=0)
counts1.columns = counts1.columns.map(lambda x: f"STRTTIME{x}")
df_p = df_p.merge(counts1, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'STRTTIME1': 'AM', 'STRTTIME2': 'PM', 'STRTTIME3': 'MIDDAY', 'STRTTIME4': 'NIGHT'}, inplace=True)

In [145]:
#this two new columns will show the time the person did their first and last trips, as is obvious in the columns' name
res =(df.sort_values(["HOUSEID","PERSONID"])
    .groupby(["HOUSEID", "PERSONID"], as_index=False)
    .agg({"STRTTIME":"first","ENDTIME":"last"})
    .rename(columns={"STRTTIME":"firsttrip_time","ENDTIME":"lasttrip_time"}))

In [146]:
# this will convert the number to integer numbers in range of 0 to 23
res['firsttrip_time'] = np.where((res['firsttrip_time'] >= 0) & (res['firsttrip_time'] <= 100), 0, 
                  np.where((res['firsttrip_time'] >= 101) & (res['firsttrip_time'] <= 200), 1,
                  np.where((res['firsttrip_time'] >= 201) & (res['firsttrip_time'] <= 300), 2,
                  np.where((res['firsttrip_time'] >= 301) & (res['firsttrip_time'] <= 400), 3,
                  np.where((res['firsttrip_time'] >= 401) & (res['firsttrip_time'] <= 500), 4,
                  np.where((res['firsttrip_time'] >= 501) & (res['firsttrip_time'] <= 600), 5,
                  np.where((res['firsttrip_time'] >= 601) & (res['firsttrip_time'] <= 700), 6,
                  np.where((res['firsttrip_time'] >= 701) & (res['firsttrip_time'] <= 800), 7,
                  np.where((res['firsttrip_time'] >= 801) & (res['firsttrip_time'] <= 900), 8,
                  np.where((res['firsttrip_time'] >= 901) & (res['firsttrip_time'] <= 1000), 9,
                  np.where((res['firsttrip_time'] >= 1001) & (res['firsttrip_time'] <= 1100), 10,
                  np.where((res['firsttrip_time'] >= 1101) & (res['firsttrip_time'] <= 1200), 11,
                  np.where((res['firsttrip_time'] >= 1201) & (res['firsttrip_time'] <= 1300), 12, 
                  np.where((res['firsttrip_time'] >= 1301) & (res['firsttrip_time'] <= 1400), 13,
                  np.where((res['firsttrip_time'] >= 1401) & (res['firsttrip_time'] <= 1500), 14,
                  np.where((res['firsttrip_time'] >= 1501) & (res['firsttrip_time'] <= 1600), 15,
                  np.where((res['firsttrip_time'] >= 1601) & (res['firsttrip_time'] <= 1700), 16,
                  np.where((res['firsttrip_time'] >= 1701) & (res['firsttrip_time'] <= 1800), 17,
                  np.where((res['firsttrip_time'] >= 1801) & (res['firsttrip_time'] <= 1900), 18,
                  np.where((res['firsttrip_time'] >= 1901) & (res['firsttrip_time'] <= 2000), 19,
                  np.where((res['firsttrip_time'] >= 2001) & (res['firsttrip_time'] <= 2100), 20,
                  np.where((res['firsttrip_time'] >= 2101) & (res['firsttrip_time'] <= 2200), 21,
                  np.where((res['firsttrip_time'] >= 2201) & (res['firsttrip_time'] <= 2300), 22,
                  np.where((res['firsttrip_time'] >= 2301) & (res['firsttrip_time'] <= 2359), 23,-9))))))))))))))))))))))))

res['lasttrip_time'] = np.where((res['lasttrip_time'] >= 0) & (res['lasttrip_time'] <= 100), 0, 
                  np.where((res['lasttrip_time'] >= 101) & (res['lasttrip_time'] <= 200), 1,
                  np.where((res['lasttrip_time'] >= 201) & (res['lasttrip_time'] <= 300), 2,
                  np.where((res['lasttrip_time'] >= 301) & (res['lasttrip_time'] <= 400), 3,
                  np.where((res['lasttrip_time'] >= 401) & (res['lasttrip_time'] <= 500), 4,
                  np.where((res['lasttrip_time'] >= 501) & (res['lasttrip_time'] <= 600), 5,
                  np.where((res['lasttrip_time'] >= 601) & (res['lasttrip_time'] <= 700), 6,
                  np.where((res['lasttrip_time'] >= 701) & (res['lasttrip_time'] <= 800), 7,
                  np.where((res['lasttrip_time'] >= 801) & (res['lasttrip_time'] <= 900), 8,
                  np.where((res['lasttrip_time'] >= 901) & (res['lasttrip_time'] <= 1000), 9,
                  np.where((res['lasttrip_time'] >= 1001) & (res['lasttrip_time'] <= 1100), 10,
                  np.where((res['lasttrip_time'] >= 1101) & (res['lasttrip_time'] <= 1200), 11,
                  np.where((res['lasttrip_time'] >= 1201) & (res['lasttrip_time'] <= 1300), 12, 
                  np.where((res['lasttrip_time'] >= 1301) & (res['lasttrip_time'] <= 1400), 13,
                  np.where((res['lasttrip_time'] >= 1401) & (res['lasttrip_time'] <= 1500), 14,
                  np.where((res['lasttrip_time'] >= 1501) & (res['lasttrip_time'] <= 1600), 15,
                  np.where((res['lasttrip_time'] >= 1601) & (res['lasttrip_time'] <= 1700), 16,
                  np.where((res['lasttrip_time'] >= 1701) & (res['lasttrip_time'] <= 1800), 17,
                  np.where((res['lasttrip_time'] >= 1801) & (res['lasttrip_time'] <= 1900), 18,
                  np.where((res['lasttrip_time'] >= 1901) & (res['lasttrip_time'] <= 2000), 19,
                  np.where((res['lasttrip_time'] >= 2001) & (res['lasttrip_time'] <= 2100), 20,
                  np.where((res['lasttrip_time'] >= 2101) & (res['lasttrip_time'] <= 2200), 21,
                  np.where((res['lasttrip_time'] >= 2201) & (res['lasttrip_time'] <= 2300), 22,
                  np.where((res['lasttrip_time'] >= 2301) & (res['lasttrip_time'] <= 2359), 23,-9))))))))))))))))))))))))
x = res.groupby(['HOUSEID','PERSONID']).sum()# this has no effect and it is done to just prevent an error

#add first trip and last trip time to the person dataframe
df_p = df_p.merge(x, how="left", left_on=['HOUSEID', 'PERSONID'], right_index=True)

In [147]:
'''df = df[df['TRVL_MIN'] < 120]# trips longer than 2 hours and longer than 30 miles are removed in order to prevent bias
df = df[df['TRPMILES'] < 30]# so are the trips longer than 30 miles
df = df[df['DWELTIME'] < 720]# and also the trips that have no trips after them for more than 12 hours(720 minutes)'''

"df = df[df['TRVL_MIN'] < 120]# trips longer than 2 hours and longer than 30 miles are removed in order to prevent bias\ndf = df[df['TRPMILES'] < 30]# so are the trips longer than 30 miles\ndf = df[df['DWELTIME'] < 720]# and also the trips that have no trips after them for more than 12 hours(720 minutes)"

In [148]:
# this makes 2 columns of mean travel time for work and shop trips 
new_df1=df.pivot_table(index=['HOUSEID', 'PERSONID'],columns='WHYTO',values='TRVL_MIN')
new_df1.fillna(0, inplace=True)

df_p = df_p.merge(new_df1, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={2: 'work_traveltime'}, inplace=True)
df_p.drop([4, 3], axis=1, inplace=True)

In [149]:
#sum of all lenght traveled by the person
new_df2=df.groupby(['HOUSEID','PERSONID'] )['TRPMILES'].sum()
df_p = df_p.merge(new_df2, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'TRPMILES': 'TRPMILES_sum'}, inplace=True)

#sum of all the time the person was traveling
new_df3=df.groupby(['HOUSEID','PERSONID'] )['TRVL_MIN'].sum()
df_p = df_p.merge(new_df3, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'TRVL_MIN': 'TRVL_MIN_sum'}, inplace=True)

In [150]:
df['TRPMILES'] /= df['TRPMILES'].max()

new_df=df.pivot_table(index=['HOUSEID', 'PERSONID'],columns='WHYTO',values='TRPMILES')
new_df.fillna(0, inplace=True)

df_p = df_p.merge(new_df, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={2: 'work_tripmile'}, inplace=True)
df_p.drop([4, 3], axis=1, inplace=True)

In [151]:
#trip miles for each person, normalized by state and urban-rural classification
s=df.groupby(['HOUSEID',  'PERSONID'])['TRPMILES'].mean()
df_p = df_p.merge(s, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'TRPMILES': 'TRPMILES_mean'}, inplace=True)

#mean traveltime of all trips
s1=df.groupby(['HOUSEID',  'PERSONID'])['TRVL_MIN'].mean()
df_p = df_p.merge(s1, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'TRVL_MIN': 'TRVL_MIN_mean'}, inplace=True)

#mean of the time the person is not traveling
s2=df.groupby(['HOUSEID',  'PERSONID'])['DWELTIME'].mean()
df_p = df_p.merge(s2, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={'DWELTIME': 'DWELTIME_mean'}, inplace=True)

In [152]:
# mean time spent  for shopping and work 
new_df2=df.pivot_table(index=['HOUSEID', 'PERSONID'],columns='WHYTO',values='DWELTIME')
new_df2.fillna(0, inplace=True)

df_p = df_p.merge(new_df2, how="left", left_on=["HOUSEID", "PERSONID"], right_index=True)
df_p.rename(columns={2: 'work_dweltime'}, inplace=True)
df_p.drop([4,3], axis=1, inplace=True)

In [153]:
# a new column is defined to show the average vehicle ownership ratio in each individual's family.(vehiclecount\householdsize)
df_p['AVG_VEH_CNT'] = ''
df_p['AVG_VEH_CNT'] = df_p['HHVEHCNT'].divide(df_p['HHSIZE'])


In [154]:
list_x = []
for i in range(0,18):
  list_x.append(i)

In [155]:
'''#setting under 18 people as np.nan
df_p['R_AGE'].replace(list_x, np.nan, inplace=True)'''

"#setting under 18 people as np.nan\ndf_p['R_AGE'].replace(list_x, np.nan, inplace=True)"

In [156]:
df_p.dropna(axis=0, inplace=True)
df_p.reset_index(drop=True, inplace=True)

In [157]:
df_p

,HOUSEID,PERSONID,R_SEX,R_AGE,wkstat,HHFAMINC,HHSIZE,HHVEHCNT,home_tpb_taz,hhwrk,...,TRPMILES_sum,TRVL_MIN_sum,-9_y,work_tripmile,TRPMILES_mean,TRVL_MIN_mean,DWELTIME_mean,-9,work_dweltime,AVG_VEH_CNT
0,140000271,14000027101,0,59,0,4,3,4,3111,3,...,13.121000,51.0,0.0,0.000246,0.000217,10.200000,85.800000,0.0,187.5,1.333333
1,140000271,14000027103,1,19,0,4,3,4,3111,3,...,16.616000,41.0,0.0,0.000000,0.000686,20.500000,272.000000,0.0,0.0,1.333333
2,140000280,14000028001,1,67,0,3,1,2,3113,1,...,56.650000,112.0,0.0,0.000771,0.000936,22.400000,154.600000,0.0,445.0,2.000000
3,140000284,14000028401,0,70,1,2,1,1,3114,0,...,32.579000,105.0,0.0,0.000000,0.000538,21.000000,39.200000,0.0,0.0,1.000000
4,140000291,14000029101,0,63,0,3,2,2,3114,1,...,19.604000,55.0,0.0,0.000727,0.000540,18.333333,195.000000,0.0,530.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32918,180256936,18025693601,1,26,4,3,1,1,742,0,...,65.419653,95.0,0.0,0.000000,0.001081,19.000000,135.000000,0.0,0.0,1.000000
32919,180256940,18025694001,0,70,1,4,2,3,2669,0,...,8.375931,45.0,0.0,0.000000,0.000138,9.000000,9.000000,0.0,0.0,1.500000
32920,180256940,18025694002,1,71,1,4,2,3,2669,0,...,40.583171,100.0,0.0,0.000000,0.000671,20.000000,77.000000,0.0,0.0,1.500000
32921,180256941,18025694101,0,60,2,3,2,3,1236,1,...,34.593945,70.0,0.0,0.000000,0.001429,35.000000,35.000000,0.0,0.0,1.500000


In [158]:
'''df_p['region_1'] = 0
df_p['region_2'] = 0
df_p['region_3'] = 1
df_p['region_4'] = 0'''

"df_p['region_1'] = 0\ndf_p['region_2'] = 0\ndf_p['region_3'] = 1\ndf_p['region_4'] = 0"

In [159]:
df_zone = pd.read_csv('./zone.csv')

In [160]:
df_zone = df_zone[['TAZ', 'MTLRTSHR','MTLRTLNG', 'ALLPKSHR', 'ALLPKLNG', 'ALLOPSHR', 'ALLOPLNG',
        'popden', 'empden']]

In [161]:
df_zone.rename(columns={'TAZ': 'home_tpb_taz'}, inplace=True)

In [162]:
df_zone

,home_tpb_taz,MTLRTSHR,MTLRTLNG,ALLPKSHR,ALLPKLNG,ALLOPSHR,ALLOPLNG,popden,empden
0,1,0.000000,45.069961,100.000003,100.000003,100.000003,99.999999,0.0,270.943396
1,2,0.000000,0.000000,64.636820,100.000004,63.209846,99.999999,0.0,0.882164
2,3,0.000000,21.237789,2.871646,53.642877,2.858598,47.685633,0.0,0.000000
3,4,23.904363,80.424722,37.056351,99.342999,26.472568,77.836342,0.0,0.027670
4,5,0.000000,18.844193,13.709852,98.654050,13.777166,98.633479,0.0,0.000000
...,...,...,...,...,...,...,...,...,...
3717,3718,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3718,3719,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3719,3720,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3720,3721,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [163]:
taz_dict = df_zone.set_index('home_tpb_taz').T.to_dict('list')

In [164]:
def taz_assigner(taz_number):
  if taz_number in taz_dict.keys():
    taz_dict[taz_number]
    MTLRTSHR = taz_dict[taz_number][0]
    MTLRTLNG = taz_dict[taz_number][1]
    ALLPKSHR = taz_dict[taz_number][2]
    ALLPKLNG = taz_dict[taz_number][3]
    ALLOPSHR = taz_dict[taz_number][4]
    ALLOPLNG = taz_dict[taz_number][5]
    popden = taz_dict[taz_number][6]
    empden = taz_dict[taz_number][7]
    return pd.Series([MTLRTSHR, MTLRTLNG,	ALLPKSHR,	ALLPKLNG,	ALLOPSHR,	ALLOPLNG,	popden,	empden])


In [165]:
df_p['MTLRTSHR'] = ''
df_p['MTLRTLNG'] = ''
df_p['ALLPKSHR'] = ''
df_p['ALLPKLNG'] = ''
df_p['ALLOPSHR'] = ''
df_p['ALLOPLNG'] = ''
df_p['popden'] = ''
df_p['empden'] = ''
df_p[['MTLRTSHR', 'MTLRTLNG',	'ALLPKSHR',	'ALLPKLNG',	'ALLOPSHR',	'ALLOPLNG',	'popden',	'empden']] = (df_p.apply(lambda row:taz_assigner(row['home_tpb_taz']),axis=1))

In [166]:
def school_edu_div(education, age):
  if age < 18:
    school = education
    return pd.Series([school,0])
  else:
    return pd.Series([0,education])

df_p['school'] = ''
df_p[['school','EDUCATION']] = (df_p.apply(lambda row:school_edu_div(row['EDUCATION'], row['R_AGE']),axis=1))

In [167]:
#df_p['Full_Time_Worker'] = ''

#defining 5 age ranges
#df_p['Full_Time_Worker'] = np.where((df_p['hours'] >= 35), 1, 0)

In [168]:
#defining 5 age ranges
df_p['R_AGE_CAT'] = np.where((df_p['R_AGE'] >= 18) & (df_p['R_AGE'] <= 24), 1,
                         np.where((df_p['R_AGE'] >= 25) & (df_p['R_AGE'] <= 34), 2,
                         np.where((df_p['R_AGE'] >= 35) & (df_p['R_AGE'] <= 50), 3,
                         np.where((df_p['R_AGE'] >= 50) & (df_p['R_AGE'] <= 65), 4,
                         np.where((df_p['R_AGE'] >= 0) & (df_p['R_AGE'] <= 17), 0,                                                                 
                                  5)))))

In [169]:
df_p['university_student'] = np.where((df_p['wkstat'] == 6) & (df_p['R_AGE'] >= 18), 1, 0)
df_p['Student_Age_5_15'] = np.where((df_p['R_AGE'] >= 5) & (df_p['R_AGE'] <= 15), 1, 0)
df_p['Student_Age_16_up'] = np.where((df_p['R_AGE'] == 16) & (df_p['R_AGE'] <= 17), 1, 0)

In [170]:
df_p

,HOUSEID,PERSONID,R_SEX,R_AGE,wkstat,HHFAMINC,HHSIZE,HHVEHCNT,home_tpb_taz,hhwrk,...,ALLPKLNG,ALLOPSHR,ALLOPLNG,popden,empden,school,R_AGE_CAT,university_student,Student_Age_5_15,Student_Age_16_up
0,140000271,14000027101,0,59,0,4,3,4,3111,3,...,0.000000,0.000000,0.000000,0.111091,0.004592,0.0,4,0,0,0
1,140000271,14000027103,1,19,0,4,3,4,3111,3,...,0.000000,0.000000,0.000000,0.111091,0.004592,0.0,1,0,0,0
2,140000280,14000028001,1,67,0,3,1,2,3113,1,...,0.000000,0.000000,0.000000,0.049211,0.004264,0.0,5,0,0,0
3,140000284,14000028401,0,70,1,2,1,1,3114,0,...,0.000000,0.000000,0.000000,0.004909,0.009082,0.0,5,0,0,0
4,140000291,14000029101,0,63,0,3,2,2,3114,1,...,0.000000,0.000000,0.000000,0.004909,0.009082,0.0,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32918,180256936,18025693601,1,26,4,3,1,1,742,0,...,99.999999,18.820135,99.999995,1.167582,9.855769,0.0,2,0,0,0
32919,180256940,18025694001,0,70,1,4,2,3,2669,0,...,22.933544,4.503875,22.874925,1.056665,0.865094,0.0,5,0,0,0
32920,180256940,18025694002,1,71,1,4,2,3,2669,0,...,22.933544,4.503875,22.874925,1.056665,0.865094,0.0,5,0,0,0
32921,180256941,18025694101,0,60,2,3,2,3,1236,1,...,0.000000,0.000000,0.000000,0.000000,0.001283,0.0,4,0,0,0


In [179]:
df_p.reset_index(inplace=True, drop=True)

In [181]:
def pattern(work, education, school):
  if work == 1:
    if education >= 1:
      pattern = '1_work_1_edu'
    else:
      pattern = '1_work'
  elif work == 2:
    pattern = '2_work'
  elif work >= 3:
    pattern = '3_work'
  elif education >= 1:
    pattern = '1_edu'
  elif school == 1:
    pattern = '1_sch'
  elif school >= 2:
    pattern = '1_sch_1_edu'
  else:
    pattern = 'none'
  return pattern

df_p['M_pattern'] = ''
df_p['M_pattern'] = (df_p.apply(lambda row:pattern(row['WORK'], row['EDUCATION'], row['school']),axis=1))

In [182]:
df_p.M_pattern.value_counts()

none            13424
1_work          11447
1_sch            3301
2_work           2652
3_work            856
1_edu             754
1_sch_1_edu       257
1_work_1_edu      232
Name: M_pattern, dtype: int64

In [ ]:
# def max_mode(transit, auto):
#   if transit == auto:
#     return 'neutral'
#   elif transit==0 and auto ==0:
#     return 'other'
#   else:
#     output = max(transit, auto)
#     if output == transit:
#       return 'transit'
#     elif output == auto:
#       return 'auto'

# df_p['pmode'] = (df_p.apply(lambda row:max_mode(row['transit'], row['auto']),axis=1))

In [ ]:
#df_p['pmode'].value_counts()

auto       18168
transit     1967
neutral     1881
Name: pmode, dtype: int64

In [ ]:
# def mode_percent(transit, auto, neutral):
#   sum = transit+ auto+ neutral
#   t_percent = transit/sum
#   a_percent = auto/sum
#   n_percent = neutral/sum
#   return pd.Series([t_percent,a_percent,n_percent])
# df_p['t_percent']=''
# df_p['a_percent']=''
# df_p['n_percent']=''
# df_p[['t_percent', 'a_percent', 'n_percent']] = (df_p.apply(lambda row:mode_percent(row['transit'], row['auto'], row['other']),axis=1))

In [183]:
df_p.to_csv('washington_data_m_n_predict6.csv')